In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [2]:
  df = fetch_datasets()['mammography']
  X, y = df.data, df.target
  le = LabelEncoder()
  y = le.fit_transform(y)
  G = len(np.unique(y))
  print(np.shape(X), G, sum(y==0)*.6*.5, sum(y==1)*.6*.5)
  X[:3]

(11183, 6) 2 3276.9 78.0


array([[ 0.23001961,  5.0725783 , -0.27606055,  0.83244412, -0.37786573,
         0.4803223 ],
       [ 0.15549112, -0.16939038,  0.67065219, -0.85955255, -0.37786573,
        -0.94572324],
       [-0.78441482, -0.44365372,  5.6747053 , -0.85955255, -0.37786573,
        -0.94572324]])

In [3]:
missing_rates = np.array([0.5, 0.6, 0.7, 0.8, 0.9])
res_all = []
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  n = np.array([-200,100])
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [4]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [5]:
leftindex = pd.MultiIndex.from_product([["50%", "60%", "70%", "80%", "90%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM          0.302$\pm$0.27  0.977$\pm$0.002   
                         LR          0.747$\pm$0.049  0.981$\pm$0.002   
                         DCT         0.055$\pm$0.112   0.97$\pm$0.004   
             sensitivity SVM         0.234$\pm$0.203  0.981$\pm$0.001   
                         LR          0.627$\pm$0.063  0.983$\pm$0.002   
                         DCT         0.055$\pm$0.067  0.969$\pm$0.004   
60%          F-1 score   SVM         0.057$\pm$0.022  0.977$\pm$0.002   
                         LR          0.767$\pm$0.022   0.98$\pm$0.002   
                         DCT          0.005$\pm$0.01  0.969$\pm$0.003   
             sensitivity SVM         0.052$\pm$0.011   0.98$\pm$0.002   
                         LR          0.651$\pm$0.029  0.982$\pm$0.002   
                         DCT         0.025$\pm$0.006  0.968$\pm$0.005   
70%          F-1 score   SVM         0.128$\pm$0.167  0.975$\pm$0.003   
                         LR           0.74$\pm$0.034   0.98$\pm$0.002   
                         DCT         0.024$\pm$0.035  0.962$\pm$0.006   
             sensitivity SVM         0.102$\pm$0.124   0.98$\pm$0.002   
                         LR          0.617$\pm$0.044  0.982$\pm$0.002   
                         DCT         0.035$\pm$0.018  0.957$\pm$0.009   
80%          F-1 score   SVM         0.117$\pm$0.075  0.973$\pm$0.003   
                         LR           0.74$\pm$0.027  0.978$\pm$0.003   
                         DCT         0.039$\pm$0.056  0.955$\pm$0.008   
             sensitivity SVM         0.087$\pm$0.042  0.978$\pm$0.002   
                         LR          0.616$\pm$0.034  0.979$\pm$0.004   
                         DCT         0.044$\pm$0.031  0.948$\pm$0.013   
90%          F-1 score   SVM          0.25$\pm$0.177  0.968$\pm$0.005   
                         LR          0.713$\pm$0.035  0.971$\pm$0.011   
                         DCT         0.015$\pm$0.033  0.935$\pm$0.021   
             sensitivity SVM         0.179$\pm$0.131  0.974$\pm$0.006   
                         LR          0.582$\pm$0.043  0.968$\pm$0.015   
                         DCT          0.03$\pm$0.018   0.91$\pm$0.036   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM         0.976$\pm$0.002  0.976$\pm$0.003   
                         LR          0.981$\pm$0.002  0.981$\pm$0.002   
                         DCT         0.978$\pm$0.001   0.97$\pm$0.004   
             sensitivity SVM          0.98$\pm$0.001  0.981$\pm$0.002   
                         LR          0.983$\pm$0.001  0.984$\pm$0.001   
                         DCT         0.979$\pm$0.002  0.968$\pm$0.004   
60%          F-1 score   SVM         0.976$\pm$0.003  0.976$\pm$0.002   
                         LR           0.98$\pm$0.003   0.98$\pm$0.002   
                         DCT         0.975$\pm$0.004  0.967$\pm$0.005   
             sensitivity SVM          0.98$\pm$0.002  0.981$\pm$0.002   
                         LR          0.982$\pm$0.002  0.982$\pm$0.001   
                         DCT         0.976$\pm$0.006  0.964$\pm$0.008   
70%          F-1 score   SVM         0.973$\pm$0.004  0.974$\pm$0.004   
                         LR          0.978$\pm$0.003   0.98$\pm$0.001   
                         DCT         0.976$\pm$0.003  0.964$\pm$0.005   
             sensitivity SVM         0.979$\pm$0.002  0.979$\pm$0.002   
                         LR          0.981$\pm$0.002  0.983$\pm$0.001   
                         DCT         0.979$\pm$0.003  0.959$\pm$0.007   
80%          F-1 score   SVM         0.967$\pm$0.004  0.971$\pm$0.004   
                         LR          0.977$\pm$0.005   0.98$\pm$0.004   
                         DCT         0.971$\pm$0.004  0.956$\pm$0.008   
             sensitivity SVM

In [6]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_mammography-high.csv", index = False)

In [7]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
50\% & F-1 score & SVM &   0.302\$\textbackslash pm\$0.27 &  0.977\$\textbackslash pm\$0.002 &  0.976\$\textbackslash pm\$0.002 &  0.976\$\textbackslash pm\$0.003 &  0.968\$\textbackslash pm\$0.004 &  0.974\$\textbackslash pm\$0.003 \\
    &             & LR &  0.747\$\textbackslash pm\$0.049 &  0.981\$\textbackslash pm\$0.002 &  0.981\$\textbackslash pm\$0.002 &  0.981\$\textbackslash pm\$0.002 &  0.966\$\textbackslash pm\$0.005 &   0.98\$\textbackslash pm\$0.002 \\
    &             & DCT &  0.055\$\textbackslash pm\$0.112 &   0.97\$\textbackslash pm\$0.004 &  0.978\$\textbackslash pm\$0.001 &   0.97\$\textbackslash pm\$0.004 &  0.905\$\textbackslash p